## 5.- Webserver con ESP8266

En este capitulo se describira como controlar salidas tales como: Motores, Led, servomotores, etc. Entradas tales como: Sensores de temperatura, humedad, distancia, etc. Para cumplir con este fin se programara al ESP8266 como un servidor que permita realizar las acciones. Este programa se denominara webserver.

El programa para el ESP8266 debe:
- Conectarse a Internet.
- Controlar la salidad.
- Programar un webserver que realice las acciones.

Para conectar el ESP8266 a una red WIFI se debe utilizar la libreria  ESP8266Wifi y para construir el webServer la libreria ESP8266WebServer, para acceder a la memoria FLASH se utilizara PROGMEN

### 5.1.- Libreria Esp8266Wifi

La librería WiFi para ESP8266  ha sido desarrollada para, conectar el soc (sistema en un chip) a una red WIFI. basándose en el SDK de ESP8266, usando nombres convencionales y la filosofía de funcionalidades generales de la librería WiFi de Arduino. 

|Nombre  |   Sintaxis                  |            Funcion                                        |
|---     |---                          |---                                                        |
|begin   |WiFi.begin(ssid,contraseña); |Constructor del objeto  WIFI                               |
|        |                             |ssid: nombre de la red WIFI                                |
|        |                             |contraseña: contraseña de la red                           |
|status  |WiFi.status()                | Presenta el estado de la conexion                         |
|        |                             |  WL_CONNECTED: indica que esta conectado                  |
|localIP |WiFi.localIP()               | Presenta la direccion de red donde se conecto             |


### 5.2- Codigo de prueba ESP8266WiFi

Se presenta el codigo de un programa que permite conectar un ESP8266 a una red. Se utiliza las funciones descriptas en la seccion anterior.
```
#include <ESP8266WiFi.h>  
void setup()  
{ Serial.begin(115200);  
Serial.println();  
WiFi.begin("nombre-red", "contraseña-red");  
Serial.print("Conectando");  
while (WiFi.status() != WL_CONNECTED)  
{  
delay(500);  
Serial.print(".");  
}  
Serial.println();  
Serial.print("Conectado, dirección IP: ");  
Serial.println(WiFi.localIP());  
}  

void loop() {}
``` 


Para poder realizar un codigo de programa reusable y facilmente modificable. Se recomienda crear tres archivos.

- internet0.ino: Donde se encuentra el programa principal.
- config.h: Donde se encuentran todas las constantes del programa.
- wifi.hpp: Donde se encuentran las funciones que se utizan para trabajae en Internet.

#### Abrir el IDE de Arduino.
  Seleccionar  Archivo -> New Sketch apretar ENTER   
  
  Aparece un sketch nuevo como el de la figura.   
  
<center><img src="img/wifi0.jpg" alt="Imagen"></img></center>  
  Seleccionar Archivo -> Save   
  
En la pestaña que se muestra en la figura, se coloca internet0 en el dialogo Nombre y  apretar el boton Guardar   

<center><img src="img/wifi1.jpg" alt="Imagen"></img></center>   

Seleccionar con un click del raton en los tres puntos.   

<center><img src="img/wifi2.jpg" alt="Imagen"></img></center>  

Aparece el dialogo que se muestra en la figura.   

<center><img src="img/wifi3.jpg" alt="Imagen"></img></center>

Seleccionar nueva pestaña, en la misma colocar config.h, repitiendo este procedimiento colocar wifi.hpp. El resultado se muestra en la siguiente figura.   

<center><img src="img/wifi4.jpg" alt="Imagen"></img></center>  


Se debe colocar en config.h el siguiente codigo
```
const char* red_WIFI = "nombre_red";
const char* clave = "clave";
```

En wifi.hpp se debe escribir el siguiente codigo
```
void conectarWIFI() {
  WiFi.begin(red_WIFI, clave);
  Serial.print("Conectando");
  while (WiFi.status() != WL_CONNECTED) {
    delay(500);
    Serial.print(".");
  }
  Serial.println();
  Serial.print("Conectado, dirección IP: ");
  Serial.println(WiFi.localIP());
}
```

En internet0.ino se debe escribir el siguiente codigo.

```
#include <ESP8266WiFi.h>
#include "config.h"
#include "wifi.hpp"
void setup()  
{
Serial.begin(115200);  
conectar_WIFI();
}  

void loop() {}
``` 
Se debe compilar y cargar el programa en el esp8266. Cuando se ejecuta, aparece en la consola, la dirección de IP, donde se conecto el dispositivo.
